In [1]:
import pandas as pd
import numpy as np
import regex as re
import os
import gensim.downloader as api
from numpy.linalg import norm
from sklearn.feature_extraction.text import CountVectorizer
import zipfile
from gensim.models import KeyedVectors
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# nltk.download('punkt')
# nltk.download('stopwords')

In [2]:
# uncomment to save the word2vec vector model
# os.makedirs('./vectors')
# vectorizer = api.load('glove-wiki-gigaword-300')
# vectorizer.save('./vectors/vectors.kv') # saving vectorizer

vectorizer = KeyedVectors.load('./vectors/vectors.kv') # loading vectorizer from local

In [3]:
with zipfile.ZipFile('Dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

In [4]:
fkdf = pd.read_csv(r'Dataset\flipkart_com-ecommerce_sample.csv', encoding= 'unicode_escape')
amdf = pd.read_csv(r'Dataset\amz_com-ecommerce_sample.csv', encoding= 'unicode_escape')

In [5]:
print(fkdf.shape)
print(amdf.shape)

(20000, 15)
(20000, 15)


In [6]:
fkdf.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [7]:
amdf.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,982,438,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32143,29121,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,991,551,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,694,325,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,208,258,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [8]:
# checking if the flipkart and amazon dataset are same or not
for i in amdf.columns:
  if sum(amdf[i] == fkdf[i]) == 20000:
    print(f'"{i}" value in both the dataset is same and at same location: TRUE')
print()
for i in amdf.columns:
  if sum(amdf[i] == fkdf[i]) != 20000:
    print(f'"{i}" value in both the dataset is same and at same location: FALSE')

"uniq_id" value in both the dataset is same and at same location: TRUE
"crawl_timestamp" value in both the dataset is same and at same location: TRUE
"product_url" value in both the dataset is same and at same location: TRUE
"pid" value in both the dataset is same and at same location: TRUE
"is_FK_Advantage_product" value in both the dataset is same and at same location: TRUE
"product_rating" value in both the dataset is same and at same location: TRUE
"overall_rating" value in both the dataset is same and at same location: TRUE

"product_name" value in both the dataset is same and at same location: FALSE
"product_category_tree" value in both the dataset is same and at same location: FALSE
"retail_price" value in both the dataset is same and at same location: FALSE
"discounted_price" value in both the dataset is same and at same location: FALSE
"image" value in both the dataset is same and at same location: FALSE
"description" value in both the dataset is same and at same location: FAL

In [9]:
print(len(amdf['pid'].unique()))
print(len(amdf['uniq_id'].unique()))

19998
20000


This means that 2 pid values are duplicate in the datset

In [10]:
print('Number of duplicate rows (FLIPKART):', sum(fkdf.duplicated()))
print('Number of duplicate rows (AMAZON):', sum(amdf.duplicated()))

Number of duplicate rows (FLIPKART): 0
Number of duplicate rows (AMAZON): 0


In [11]:
duplicate = fkdf[fkdf.duplicated('pid')]
duplicate

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
7769,82f424a90d630acc188fcf267160a1fe,2016-06-16 06:18:58 +0000,http://www.flipkart.com/hako-60w-magsafe1-lapt...,Hako 60W Magsafe1 Laptop Charger For APPLE Mac...,"[""Computers >> Laptop Accessories >> Laptop Ad...",ACCEJ6TESY7AFT5W,2399.0,1799.0,"[""http://img6a.flixcart.com/image/laptop-adapt...",False,Key Features of Hako 60W Magsafe1 Laptop Charg...,No rating available,No rating available,Hako,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
9419,2867c80d88e3693c883463e6358e6cc5,2016-05-10 11:57:25 +0000,http://www.flipkart.com/timberlake-slim-fit-wo...,TIMBERLAKE Slim Fit Fit Women's Blue Jeans,"[""Clothing >> Women's Clothing >> TIMBERLAKE W...",JEAEGE8Q8GXYFTGU,1499.0,1099.0,"[""http://img5a.flixcart.com/image/jean/g/p/z/a...",False,Key Features of TIMBERLAKE Slim Fit Fit Women'...,No rating available,No rating available,TIMBERLAKE,"{""product_specification""=>[{""key""=>""Brand Colo..."


In [12]:
# printing the first element of row
print(amdf['product_name'][0])
print(amdf['product_category_tree'][0])
print(amdf['image'][0])
print(amdf['description'][0])
print(amdf['brand'][0])
print(amdf['product_specifications'][0])

Alisha Solid Women's Cycling Shorts
["Clothing >> Women's Clothing >> Lingerie, Sleep & Swimwear >> Shorts >> Alisha Shorts >> Alisha Solid Women's Cycling Shorts"]
["http://img5a.flixcart.com/image/short/u/4/a/altht-3p-21-alisha-38-original-imaeh2d5vm5zbtgg.jpeg", "http://img5a.flixcart.com/image/short/p/j/z/altght4p-26-alisha-38-original-imaeh2d5kbufss6n.jpeg", "http://img5a.flixcart.com/image/short/p/j/z/altght4p-26-alisha-38-original-imaeh2d5npdybzyt.jpeg", "http://img5a.flixcart.com/image/short/z/j/7/altght-7-alisha-38-original-imaeh2d5jsz2ghd6.jpeg"]
Key Features of Alisha Solid Women's Cycling Shorts Cotton Lycra Navy, Red, Navy,Specifications of Alisha Solid Women's Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Women's Fabric Care Gentle Machine Wash in Lukewarm Water, Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shorts
Alisha
{"product_specifica

In [13]:
stop_words = set(stopwords.words('english')) # stop words from nltk library

In [14]:
def tokenizer(sent):
  '''
  Converts sentence into token or words.'''
  clean_sent = []
  sent = sent.lower() # lower case the sentence
  ptrn = re.compile(r'[^a-z0-9]') # regex pattern for removing anything other than aplhabets and numbers
  
  for token in word_tokenize(sent): # nltk tokenizer
    # print(token)
    if token not in stop_words and re.search(ptrn, token) == None:
      clean_sent.append(token)
  
  return clean_sent

In [15]:
def col_tokenizer(df, feature):
  '''
  Apply tokenizer function to given dataset and feature.'''
  new_feature = 'clean_'+feature
  df[new_feature] = df[feature].apply(tokenizer)

In [16]:
col_tokenizer(fkdf, 'product_name')
# fkdf.head()

In [17]:
col_tokenizer(amdf, 'product_name')
# amdf.head()

In [18]:
def sent_vec(sent):
  '''
  Converts sentence(tokenized) into vectors. Returns the average of vector value of each word.'''
  counter = 1
  missed_words = []
  
  vector_size = vectorizer.vector_size  
  wv_res = np.zeros(vector_size)  # initialising resultant word vector of same size as that of vectorizer

  for word in sent:
    if word in vectorizer:
      counter += 1
      wv_res += vectorizer[word]  # summing the vector values
    else:
      missed_words.append(word)

  missed_sent = ' '.join(missed_words)  # converting missed_words into a sentence
  wv_res = wv_res/counter   # averaging the vector value of a sentence
  
  return wv_res, missed_sent


In [19]:
def col_vectorizer(df, feature):
  '''
  Apply sent_vec to given dataset and feature.'''
  new_feature1 = 'vec'+ feature.split('clean')[1]
  new_feature2 = 'miss'+ feature.split('clean')[1]
  word_vectors = []
  missed_sent = []
  for sent in df[feature]:
    wv_res, missed = sent_vec(sent)
    word_vectors.append(wv_res)
    missed_sent.append(missed)

  df[new_feature1] = word_vectors
  df[new_feature2] = missed_sent
  

In [20]:
col_vectorizer(fkdf, 'clean_product_name')
fkdf.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,clean_product_name,vec_product_name,miss_product_name
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...","[alisha, solid, women, cycling, shorts]","[0.0025558344398935637, -0.06585653397875528, ...",
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...","[fabhomedecor, fabric, double, sofa, bed]","[0.038354798406362536, 0.057540607452392575, 4...",fabhomedecor
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""...","[aw, bellies]","[0.004576663176218669, -0.19613665963212648, -...",
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...","[alisha, solid, women, cycling, shorts]","[0.0025558344398935637, -0.06585653397875528, ...",
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...","[sicons, purpose, arnica, dog, shampoo]","[-0.11778400242328643, 0.3135760009288788, -0....",sicons


In [21]:
col_vectorizer(amdf, 'clean_product_name')
amdf.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,clean_product_name,vec_product_name,miss_product_name
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,982,438,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...","[alisha, solid, women, cycling, shorts]","[0.0025558344398935637, -0.06585653397875528, ...",
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32143,29121,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...","[fabhomedecor, fabric, double, sofa, bed]","[0.038354798406362536, 0.057540607452392575, 4...",fabhomedecor
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,991,551,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""...","[aw, bellies]","[0.004576663176218669, -0.19613665963212648, -...",
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,694,325,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...","[alisha, solid, women, cycling, shorts]","[0.0025558344398935637, -0.06585653397875528, ...",
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,208,258,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...","[sicons, purpose, arnica, dog, shampoo]","[-0.11778400242328643, 0.3135760009288788, -0....",sicons


In [22]:
def cosine_similarity(A, B):
  '''
  Finds cosine similarity between two vectors.'''
  return np.dot(A,B)/(norm(A)*norm(B))

# Please Enter Test String which needs to be found in Amazon dataset.

In [23]:
test_str = input('Enter the product that needs to be searched... \n')
print(test_str)

Wineberry Women's Leggings


In [24]:
# test_str = "Wineberry Women's Leggings"
test_str = tokenizer(test_str)
test_vec, missed_sent = sent_vec(test_str)
# print(missed_sent)

### Dataset Search

In [25]:
def vector_based_similarity(df, test_vec):
    '''
    Find cosine similarity of test vector and vectors present in dataset. Returns the top 80 percentile products. 
    Here rank = cosine similarity'''
    df['rank'] = df['vec_product_name'].apply(lambda x: cosine_similarity(x, test_vec))
    match_sorted = df.sort_values('rank', ascending=False)
    match_80 = match_sorted[match_sorted['rank'] > 0.8]
    return match_80

In [26]:
def miss_word_similarity(df, missed_sent):
    '''
    Find cosine similarity of words which are not present in vectorizer vocabulary. It Bag of Words technique on the missing words.'''
    miss_word_sim = [missed_sent] + df['miss_product_name'].tolist()
    count_vectorizer = CountVectorizer().fit_transform(miss_word_sim)
    count_vectors = count_vectorizer.toarray()
    
    word_sim = np.array([cosine_similarity(count_vectors[0], i) for i in count_vectors])
    word_sim_80 = np.where(word_sim > 0.80)[0]
    word_sim_75 = np.where(word_sim > 0.75)[0]
    word_sim_70 = np.where(word_sim > 0.70)[0]
    word_sim_50 = np.where(word_sim > 0.50)[0]
    if len(word_sim_80) > 1:
        row_inds = np.delete(word_sim_80, 0)
    elif len(word_sim_75) > 1:
        row_inds = np.delete(word_sim_75, 0)
    elif len(word_sim_70) > 1:
        row_inds = np.delete(word_sim_70, 0)
    else:
        print('PRODUCT NOT FOUND'.center(90, ' '))
        print('**SIMILAR PRODUCTS**'.center(90, ' '))
        row_inds = np.where(df['rank'] == df['rank'].iloc[0])[0]
    row_inds -= 1

    result = df.iloc[row_inds, :]
    return result

In [27]:
def find_similar_product(df, test_vec, missed_sent):
    '''
    Apply vector_based_similarity and miss_word_similarity function.'''
    match_80 = vector_based_similarity(fkdf, test_vec=test_vec)
    if len(missed_sent) != 0: 
        return miss_word_similarity(match_80, missed_sent=missed_sent)
    else:
        return match_80

In [28]:
fk_res = find_similar_product(fkdf, test_vec, missed_sent)
fk_res.sort_values('discounted_price', ascending=True, inplace=True)
am_res = find_similar_product(amdf, test_vec, missed_sent) 
am_res.sort_values('discounted_price', ascending=True, inplace=True)

d:\Desktop\ShackLab\conda_env\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


### Final Result

In [29]:
dic = {
    'Product name in Flipkart' : fk_res['product_name'], 
    'Retail price in Flipkart' : fk_res['retail_price'],
    'Discounted price in Flipkart' : fk_res['discounted_price'],
    'Product name in Amazon' : am_res['product_name'], 
    'Retail price in Amazon' : am_res['retail_price'],
    'Discounted price in Amazon' : am_res['discounted_price'],
}

result = pd.DataFrame(data=dic)
result

,Product name in Flipkart,Retail price in Flipkart,Discounted price in Flipkart,Product name in Amazon,Retail price in Amazon,Discounted price in Amazon
7841,Wineberry Women's Leggings,749.0,549.0,Wineberry Women's Leggings,749.0,549.0
7842,Wineberry Women's Leggings,749.0,549.0,Wineberry Women's Leggings,749.0,549.0
7838,Wineberry Women's Leggings,749.0,549.0,Wineberry Women's Leggings,749.0,549.0
